<a href="https://colab.research.google.com/github/ioradlin/NAT_project/blob/Encoder/fast_align_BPE_corpus%26vocab%26fertility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IWSLT

#### data load & prepare

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("bbaaaa/iwslt14-de-en-preprocess")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

## BPE

In [3]:
## 빠른 txt 작업

def to_txt(x, name):
  with open(name, 'w') as file:
      file.write(x)

def read_txt(name):
  with open(name, 'r') as file:
      temp= file.read()
  return temp

In [4]:
en=[]
de=[]

for line in dataset['train']['translation']:
  en.append(line['en'])
  de.append(line['de'])

In [5]:
# corpus 준비

de_txt= '\n'.join(de)
to_txt(de_txt, 'de.txt') #20MB

en_txt= '\n'.join(en)
to_txt(en_txt, 'en.txt') #17MB

In [6]:
!pip install subword-nmt

#### de에 BPE 적용

참고:

원문으로 되돌리는법 text.replace('@@ ', '').replace('</w>', ' ').strip()

In [7]:
!subword-nmt learn-bpe -s 37000 < de.txt > bpe_de #bpe_de에 1000개의 merge(split) 규칙이 담김

100% 37000/37000 [03:17<00:00, 187.03it/s]


In [8]:
!subword-nmt apply-bpe -c bpe_de < de.txt > bpe_de.txt

In [9]:
bpe_de = read_txt('bpe_de.txt')

bpe_de[:1000]

'und was menschliche gesundheit ist , kann auch ziemlich kompliziert sein .\nund diese zwei zusammen zu bringen , erscheint vielleicht wie eine gewaltige aufgabe . aber was ich ihnen zu sagen versuche ist , dass es trotz dieser komplexität einige einfache themen gibt , von denen ich denke , wenn wir diese verstehen , können wir uns wirklich weiter entwickeln .\nund diese einfachen themen sind eigentlich keine komplexen wissenschaftlichen zusammenhänge , sondern tatsachen , die wir alle gut kennen .\nund ich werde mit dieser hier anfangen : wenn die mama nicht glücklich ist , ist keiner glücklich .\nkennen wir das nicht alle ? das haben wir alle schon erlebt .\nwenn wir das nehmen und darauf aufbauen , dann können wir einen schritt weiter gehen : wenn das meer nicht glücklich ist , ist keiner glücklich .\ndarum geht es in meinem vortrag .\nwir machen das meer auf viele verschiedene arten ziemlich unglücklich .\ndas ist ein bild der can@@ ner@@ y row von 1932 .\ndamals war die can@@ ner@

#### build_vocab

만들어버리고 load 하는게 편할듯

In [10]:
src_vocab = {'[PAD]': 0,'[UNK]': 1}
k=2

for token in bpe_de.split():
  if token not in src_vocab:
    src_vocab[token]= k
    k+=1

In [11]:
len(src_vocab)

36267

#### 영어도

In [12]:
!subword-nmt learn-bpe -s 37000 < en.txt > bpe_en

100% 37000/37000 [01:35<00:00, 387.87it/s] 


In [13]:
!subword-nmt apply-bpe -c bpe_en < en.txt > bpe_en.txt

In [14]:
bpe_en = read_txt('bpe_en.txt')

bpe_en[:1000]

'and it can be a very complicated thing , what human health is .\nand bringing those two together might seem a very daunting task , but what i &apos;m going to try to say is that even in that complexity , there &apos;s some simple themes that i think , if we understand , we can really move forward .\nand those simple themes aren &apos;t really themes about the complex science of what &apos;s going on , but things that we all pretty well know .\nand i &apos;m going to start with this one : if momma ain &apos;t happy , ain &apos;t nobody happy .\nwe know that , right ? we &apos;ve experienced that .\nand if we just take that and we build from there , then we can go to the next step , which is that if the ocean ain &apos;t happy , ain &apos;t nobody happy .\nthat &apos;s the theme of my talk .\nand we &apos;re making the ocean pretty unhappy in a lot of different ways .\nthis is a shot of cannery row in 1932 .\ncannery row , at the time , had the biggest industrial canning operation on th

In [15]:
corpus = bpe_en

tokens = corpus.split()  # Split the text into tokens (assuming tokens are separated by whitespace)
tokens_with_ampersand = [token for token in tokens if '&' in token]

print("Tokens containing '&' character:")
print(tokens_with_ampersand)


Tokens containing '&' character:
['&apos;m', '&apos;s', '&apos;t', '&apos;s', '&apos;m', '&apos;t', '&apos;t', '&apos;ve', '&apos;t', '&apos;t', '&apos;s', '&apos;re', '&apos;s', '&quot;', '&quot;', '&quot;', '&quot;', '&quot;', '&apos;s', '&apos;re', '&quot;', '&quot;', '&apos;t', '&apos;t', '&apos;re', '&apos;d', '&apos;t', '&apos;s', '&apos;t', '&apos;ll', '&apos;s', '&apos;s', '&apos;re', '&apos;s', '&apos;s', '&apos;s', '&apos;s', '&apos;re', '&apos;s', '&apos;t', '&apos;s', '&apos;s', '&apos;s', '&apos;s', '&apos;t', '&apos;s', '&quot;', '&quot;', '&apos;m', '&apos;s', '&apos;s', '&apos;re', '&apos;t', '&apos;s', '&apos;re', '&apos;t', '&apos;s', '&apos;s', '&apos;s', '&apos;ve', '&apos;s', '&apos;s', '&apos;s', '&apos;ve', '&quot;', '&quot;', '&quot;', '&apos;t', '&quot;', '&apos;t', '&apos;t', '&apos;re', '&apos;t', '&apos;s', '&apos;s', '&apos;s', '&apos;s', '&apos;s', '&apos;t', '&apos;re', '&apos;t', '&apos;s', '&apos;s', '&apos;s', '&apos;s', '&apos;t', '&apos;s', '&apos;s'

In [16]:
tgt_vocab = {'[PAD]': 0,'[UNK]': 1}
k=2

for token in bpe_en.split():
  if token not in tgt_vocab:
    tgt_vocab[token]= k
    k+=1

In [17]:
len(tgt_vocab)

33877

#### save bpe vocabs

In [18]:
import pickle

with open('src_vocab_30000.pkl', 'wb') as file:
    pickle.dump(src_vocab, file)

with open('tgt_vocab_30000.pkl', 'wb') as file:
    pickle.dump(tgt_vocab, file)

## 각각 0.1MB

## fast align

In [19]:
bpe_de_split= bpe_de.split('\n')
bpe_en_split= bpe_en.split('\n')

## 길이 확인
len(bpe_de_split)==len(dataset['train']['translation'])==len(bpe_en_split)

True

In [20]:
bpe_de_split[0]

'und was menschliche gesundheit ist , kann auch ziemlich kompliziert sein .'

In [21]:
## fast align input 전처리

def for_fastalign(bpe_de_split, bpe_en_split):
  for_align= []

  for de, en in zip(bpe_de_split, bpe_en_split):
    for_align.append(de+' ||| '+en)

  return '\n'.join(for_align)

In [22]:
fastalign_input= for_fastalign(bpe_de_split, bpe_en_split)
to_txt(fastalign_input, 'train.txt')
fastalign_input[:1000]

In [25]:
%cd /content/
!git clone https://github.com/clab/fast_align.git
!sudo apt-get install libgoogle-perftools-dev libsparsehash-dev
%cd fast_align
!mkdir build
%cd build
!cmake ..
!make
%cd /content/

/content
Cloning into 'fast_align'...
remote: Enumerating objects: 213, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 213 (delta 2), reused 4 (delta 2), pack-reused 204
Receiving objects: 100% (213/213), 70.68 KiB | 769.00 KiB/s, done.
Resolving deltas: 100% (110/110), done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libunwind-dev
The following NEW packages will be installed:
  libgoogle-perftools-dev libsparsehash-dev libunwind-dev
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 2,424 kB of archives.
After this operation, 10.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libunwind-dev amd64 1.3.2-2build2.1 [1,883 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgoogle-perftools-dev amd64 2.9.1-0ubuntu3 [470 kB]
G

In [26]:
## ✰✰✰✰✰ .align 읽어서 feritility label로 변환 ✰✰✰✰✰

from collections import Counter

def read_align(name):
  counts= []

  with open(name, 'r') as file:
      for line in file:
          line = line.strip()

          #pairs: 'a list' of 'n-n's
          pairs = line.split()

          #left_numbers: ['0', '0', '2', '1', ...
          left_numbers = [pair.split('-')[0] for pair in pairs]

          #Counter({'14': 5, '3': 4, '2': 3,...
          count = Counter(left_numbers)


          # print(left_numbers, count)
          # break

          #count: [1, 1, 3, 4, 1, 0, 0, 0, ...
          count= [count.get(str(i), 0) for i in range(max(map(int, count.keys())) + 1)]
          counts.append(count)

          ## count sum check (다르면 fertility label이 될수 없음)
          if len(left_numbers)!=sum(count):
            print('what!?', len(left_numbers),sum(count))
            break

      return counts

#### fertility label 생성

In [27]:
## txt 파일 하나에 적용 (.txt와 .align 이름만 수정)
!/content/fast_align/build/fast_align -i train.txt -d -o -v > train.align

ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 
.................................................. [50000]
.................................................. [100000]
.................................................. [150000]
..........
expected target length = source length * 1.02636
ITERATION 1
.................................................. [50000]
.................................................. [100000]
.................................................. [150000]
..........
  log_e likelihood: -6.89002e+07
  log_2 likelihood: -9.9402e+07
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.169126
       size counts: 2727
ITERATION 2
.................................................. [50000]
.................................................. [100000]
.................................................. [150000]
..........
  log_e likelihood: -1.78734e+07
  log_2 likelihood: -2.57859e+07
     cross entropy: 7.75567
        perplexity: 216.11

(Without BPE) 기본적으로 final perplexity가 40 정도인데 괜찮은 것인가..?

(With BPE 10000) 와 첫 perplexity가 1e+09 ㅋㅋㅋㅋ final은 58인데??

(With BPE 30000) final perplexity *44*

In [28]:
file_path = 'train.align'  # Replace this with the actual file path
aligned=[]
with open(file_path, 'r', encoding='utf-8') as file:
    # Read the first three lines

    for _ in range(len(de)):
        line = file.readline().strip()  # Read a line and remove leading/trailing whitespaces
        aligned.append(line)

In [29]:
## fast align을 다 돌렸다면 여기로
train_fertility= read_align('train.align')

In [30]:
train_fertility[0] #얜 숫자가 4-> 3으로 바껴있어 ㄷㄷ 뭔데...

[1, 1, 1, 1, 4, 1, 2, 0, 1, 1, 0, 1]

In [31]:
train_fertility[4]

[1, 1, 1, 0, 0, 2, 0, 1, 1, 0, 1, 1, 1]

In [32]:
aligned[4] # en 파트에서 3은 실종됐음??? ㄷㄷ

'1-0 0-1 2-2 5-4 5-5 8-6 7-7 11-8 10-9 12-10'

#### BPE 없는 버전의 align 이슈 확인

In [99]:
fastalign_input= for_fastalign(de, en)
to_txt(fastalign_input, 'not_bpe_train.txt')
fastalign_input[:1000]

'und was menschliche gesundheit ist , kann auch ziemlich kompliziert sein . ||| and it can be a very complicated thing , what human health is .\nund diese zwei zusammen zu bringen , erscheint vielleicht wie eine gewaltige aufgabe . aber was ich ihnen zu sagen versuche ist , dass es trotz dieser komplexität einige einfache themen gibt , von denen ich denke , wenn wir diese verstehen , können wir uns wirklich weiter entwickeln . ||| and bringing those two together might seem a very daunting task , but what i &apos;m going to try to say is that even in that complexity , there &apos;s some simple themes that i think , if we understand , we can really move forward .\nund diese einfachen themen sind eigentlich keine komplexen wissenschaftlichen zusammenhänge , sondern tatsachen , die wir alle gut kennen . ||| and those simple themes aren &apos;t really themes about the complex science of what &apos;s going on , but things that we all pretty well know .\nund ich werde mit dieser hier anfangen

In [100]:
## txt 파일 하나에 적용 (.txt와 .align 이름만 수정)
!/content/fast_align/build/fast_align -i not_bpe_train.txt -d -o -v > not_bpe_train.align

ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 
.................................................. [50000]
.................................................. [100000]
.................................................. [150000]
..........
expected target length = source length * 1.06991
ITERATION 1
.................................................. [50000]
.................................................. [100000]
.................................................. [150000]
..........
  log_e likelihood: -6.78838e+07
  log_2 likelihood: -9.79356e+07
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.16941
       size counts: 2426
ITERATION 2
.................................................. [50000]
.................................................. [100000]
.................................................. [150000]
..........
  log_e likelihood: -1.733e+07
  log_2 likelihood: -2.50019e+07
     cross entropy: 7.63248
        perplexity: 198.429


In [101]:
file_path = 'not_bpe_train.align'
nb_aligned=[]
with open(file_path, 'r', encoding='utf-8') as file:
    # Read the first three lines

    for _ in range(len(de)):
        line = file.readline().strip()  # Read a line and remove leading/trailing whitespaces
        nb_aligned.append(line)

#### fertility check example

wir -> 1 단어 (we)

durchlaufen -> 2단어 (go through)

initiatiohnsrituale -> 2단어 (initiation rites)

. -> 1단어(.)

In [33]:
train_fertility[0]

[1, 1, 1, 1, 4, 1, 2, 0, 1, 1, 0, 1]

In [39]:
aligned[0]

'0-0 4-1 6-2 6-3 4-4 8-5 9-6 4-7 5-8 1-9 2-10 3-11 4-12 11-13'

In [40]:
maxv = 0
temp=[]
for line in aligned:
    sources = [int(a.split('-')[0]) for a in line.split()]
    try:
        max_src = max(sources)
        outputs = [0 for _ in range(max_src+1)]
        for s in sources:
            outputs[s] += 1
        if max(outputs) > maxv:
            maxv = max(outputs) # maxv가 왜 필요하지??
        temp.append(' '.join([str(s) for s in outputs]))
    except Exception:
        print('!?') ##한번도 발생안함. 그냥 만의 하나를 위해 넣은듯


#### Fast Align fix 삽질을 시작한다..

In [42]:
temp[:3]

['1 1 1 1 4 1 2 0 1 1 0 1',
 '1 1 1 1 0 1 1 1 1 0 1 0 2 0 1 1 1 1 2 1 2 1 0 2 0 2 0 1 1 1 1 2 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 2 0 1',
 '1 1 1 3 1 1 1 1 2 1 1 2 1 1 1 1 1 2 1 1']

In [50]:
len_bpe_en = [len(line.split()) for line in bpe_en_split]
len_bpe_de = [len(line.split()) for line in bpe_de_split]
len_ft = [len(line.split()) for line in temp]
ft_sum = [sum( list(map(int, line.split()) ))for line in temp]

no_de=[] #de len != ft len
no_en=[] #en len != ft sum

i=0
for ld, lf, le, fs in zip(len_bpe_de, len_ft, len_bpe_en, ft_sum):
  if ld!=lf:
    no_de.append(i)
  if le!=fs:
    no_en.append(i)
  i+=1


print(f"de 와 ft의 길이 불일치: {len(no_de)},\n en과 ft sum의 불일치: {len(no_en)}")

이 수는 어쩌면.. bpe tokenized된 케이스들이 아닐까..

In [55]:
golbaeng= [line.count('@') for line in bpe_de_split]

In [60]:
import numpy as np
golb = np.array(golbaeng)

In [61]:
sum(golb!=0) # 아 상관있는거같다...

77622

In [62]:
golb_idx = np.nonzero(golb)[0]

@@ 포함된 인덱스, de ft 불일치, en ft 불일치

In [64]:
golb_idx[:10]

array([ 8,  9, 10, 11, 12, 14, 16, 23, 28, 29])

In [65]:
no_de[:10]

[97, 153, 243, 549, 623, 698, 822, 970, 985, 1081]

In [66]:
no_en[:10]

[1, 2, 4, 5, 6, 11, 12, 13, 14, 16]

ㅁ머임... 다 다른데? ㄷㄷ en이 심각하니까 얘부터 봐보자..

#### en!=ft 이슈 원인 파악

In [85]:
def check(i):
  print("len(de)와 len(ft): ", len_bpe_de[i], len_ft[i])
  print("len(en)과 sum(ft): ", len_bpe_en[i], ft_sum[i])
  print(aligned[i],'\n')
  print(bpe_de_split[i])

  print(bpe_en_split[i])

In [86]:
check(1)

len(de)와 len(ft):  50 50
len(en)과 sum(ft):  47 46
0-0 5-1 1-2 2-3 3-4 8-5 7-6 10-7 12-9 12-10 6-11 14-12 15-13 16-14 20-15 17-16 18-17 20-18 18-19 19-20 21-21 23-22 25-23 25-24 23-25 27-26 32-27 31-28 31-29 28-30 29-31 30-32 34-33 35-34 36-35 37-36 38-37 39-38 41-39 42-40 44-41 43-42 46-43 47-44 47-45 49-46 

und diese zwei zusammen zu bringen , erscheint vielleicht wie eine gewaltige aufgabe . aber was ich ihnen zu sagen versuche ist , dass es trotz dieser komplexität einige einfache themen gibt , von denen ich denke , wenn wir diese verstehen , können wir uns wirklich weiter entwickeln .
and bringing those two together might seem a very daunting task , but what i &apos;m going to try to say is that even in that complexity , there &apos;s some simple themes that i think , if we understand , we can really move forward .


Fast Align이 틀린게 아니긴했네.. 원문에 없던 very가 영어에서 갑자기 생기는건 align 못하긴 하지

In [87]:
check(2) #9 the, 16 on 없음 (의외의 사실은... &apos;t 같은건 매칭 잘했다는 점...)

len(de)와 len(ft):  20 20
len(en)과 sum(ft):  27 25
0-0 1-1 2-2 3-3 4-4 6-5 5-6 3-7 3-8 7-10 8-11 8-12 9-13 11-14 10-15 13-17 11-18 12-19 14-20 15-21 16-22 17-23 17-24 18-25 19-26 

und diese einfachen themen sind eigentlich keine komplexen wissenschaftlichen zusammenhänge , sondern tatsachen , die wir alle gut kennen .
and those simple themes aren &apos;t really themes about the complex science of what &apos;s going on , but things that we all pretty well know .


In [89]:
no_deen= list(set(no_de)&set(no_en))

In [90]:
#둘다 문제인 것은 몇개..??
len(no_deen)

1091

In [92]:
no_deen[:10]

[34821, 157701, 131080, 63498, 53259, 137229, 34830, 28686, 43027, 49174]

In [93]:
check(34821) #4, 8 이없음.

len(de)와 len(ft):  12 11
len(en)과 sum(ft):  14 12
0-0 1-1 1-2 1-3 5-5 5-6 6-7 4-9 6-10 7-11 10-12 10-13 

und nach einer weile schaute ich in mein ei@@ mer@@ chen –
and then , eventually , i got around to looking in my bucket .


일단 len(de), len(ft) 불일치는 쉽게 해결할수있을거같고 (len(de) 만큼의 list에 각 등장 횟수를 더하면 됨)

len(en), len(de)는 ....

4(,)는.. 모르겠고 8(schaute->looking) 이건 다른 align tool 쓰면 잘할수 있을거같기도

####not_bpe_train의 경우

In [103]:
temp=[]
for line in nb_aligned:
    sources = [int(a.split('-')[0]) for a in line.split()]
    try:
        max_src = max(sources)
        outputs = [0 for _ in range(max_src+1)]
        for s in sources:
            outputs[s] += 1
        temp.append(' '.join([str(s) for s in outputs]))
    except Exception:
        print('!?') ##한번도 발생안함

In [105]:
len_en = [len(line.split()) for line in en]
len_de = [len(line.split()) for line in de]
len_ft = [len(line.split()) for line in temp]
ft_sum = [sum( list(map(int, line.split()) ))for line in temp]

no_de=[] #de len != ft len
no_en=[] #en len != ft sum

i=0
for ld, lf, le, fs in zip(len_de, len_ft, len_en, ft_sum):
  if ld!=lf:
    no_de.append(i)
  if le!=fs:
    no_en.append(i)
  i+=1


print(f"de 와 ft의 길이 불일치: {len(no_de)},\n en과 ft sum의 불일치: {len(no_en)}")

de 와 ft의 길이 불일치: 1886,
 en과 ft sum의 불일치: 73566




결론

 1. bpe -> align 의 경우 문제가 어떤지 규명 안한 상태

2. 그러나 bpe 하든 안하든 불일치 있음 (매칭 안되는 단어는 스킵하기 때문에)

3. 따라서 fertility를 만들려면 일부 매칭을 강제할 필요가 있음

#### export (pickle)

In [35]:
import pickle

with open('train_ft_bpe_30000.pkl', 'wb') as file: #8MB
    pickle.dump(train_fertility, file)